# Synthesis S4 (random 4)

## 1. stitch frame

In [ ]:
from utils import stitch_frames
import os
import random

dataset_dir = '/mnt/data1/jiali/avsbench_data/Single-source/s4_data/visual_frames'

for split in ['test', 'train', 'val']:
    base_dir = os.path.join(dataset_dir, split)
    entries = os.listdir(base_dir)
    categories = [entry for entry in entries if '.DS_Store' not in entry]
    for temp_cate in categories:
        video_list = os.listdir(os.path.join(base_dir, temp_cate))
        for temp_video in video_list:
            input_video_path = os.path.join(base_dir, temp_cate, temp_video)

            stitch_frames(base_dir, input_video_path, split, save_floder_name = '/avsbench_synthesis_visual_random4/', num_with_audio_and_mask=random.randint(1, 4))

## 2. extract audio log-mel feature

In [ ]:
import pickle
import os
from utils import extract_log_mel_features

dataset_dir = '/mnt/data1/jiali/avsbench_synthesis_visual_random4/Single-source/s4_data/audio_wav'

for split in ['test', 'train', 'val']:
    base_dir = os.path.join(dataset_dir, split)
    entries = os.listdir(base_dir)
    categories = [entry for entry in entries if '.DS_Store' not in entry]
    for temp_cate in categories:
        print(split, temp_cate)
        audio_list = os.listdir(os.path.join(base_dir, temp_cate))
        for temp_audio in audio_list:
            input_audio_path = os.path.join(base_dir, temp_cate, temp_audio)
            log_mel_spectrogram = extract_log_mel_features(input_audio_path, n_mels=128, n_fft=2048, hop_length=512)
            
            # Create the directory for saving if it doesn't exist
            save_path1 = input_audio_path.replace('/audio_wav/', '/audio_log_mel/')
            save_path = save_path1.replace('.wav', '.pkl')
            
            # Save the features
            with open(save_path, 'wb') as f:  # Changed 'w' to 'wb' for binary write
                pickle.dump(log_mel_spectrogram, f)

# Synthesis MS3 (random 4)

## 1. stitch frame

In [ ]:
import os
import random
from utils import stitch_frames_ms3
import pandas as pd

def get_split_from_video_id(csv_file, video_id):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Search for the row with the given video_id
    row = df[df['video_id'] == video_id]
    
    # If the row is found, return the split value
    if not row.empty:
        return row['split'].values[0]
    else:
        return None


base_dir = '/mnt/data1/jiali/avsbench_synthesis_visual_random4/Multi-sources/ms3_data/visual_frames'
csv_file = '/mnt/data1/jiali/avsbench_synthesis_visual_random4/Multi-sources/ms3_meta_data.csv'

video_list = os.listdir(base_dir)
for temp_video in video_list:
    input_video_path = os.path.join(base_dir, temp_video)
    split = get_split_from_video_id(csv_file, temp_video) 
    stitch_frames_ms3(base_dir, input_video_path, split, csv_file, save_floder_name = '/avsbench_synthesis_visual_random4/',  num_with_audio_and_mask=random.randint(1, 4))

## 2. extract audio log-mel feature

In [ ]:
import pickle
import os
from utils import extract_log_mel_features

dataset_dir = '/mnt/data1/jiali/avsbench_synthesis_visual_random4/Multi-sources/ms3_data/audio_wav'

for split in ['test', 'train', 'val']:
    base_dir = os.path.join(dataset_dir, split)
    entries = os.listdir(base_dir)
    categories = [entry for entry in entries if '.DS_Store' not in entry]
    for temp_cate in categories:
        print(split, temp_cate)
        audio_list = os.listdir(os.path.join(base_dir, temp_cate))
        for temp_audio in audio_list:
            input_audio_path = os.path.join(base_dir, temp_cate, temp_audio)
            log_mel_spectrogram = extract_log_mel_features(input_audio_path, n_mels=128, n_fft=2048, hop_length=512)
            
            # Create the directory for saving if it doesn't exist
            save_path1 = input_audio_path.replace('/audio_wav/', '/audio_log_mel/')
            save_path = save_path1.replace('.wav', '.pkl')
            
            # Save the features
            with open(save_path, 'wb') as f:  # Changed 'w' to 'wb' for binary write
                pickle.dump(log_mel_spectrogram, f)